In [0]:
DECLARE catalog = 'dev';
DECLARE silver_schema = '02_silver';
DECLARE gold_schema = '03_gold_product';

In [0]:
CREATE CATALOG IF NOT EXISTS IDENTIFIER(catalog);
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(catalog || '.' || gold_schema);

In [0]:
CREATE OR REPLACE VIEW IDENTIFIER(catalog || '.' || gold_schema || '.product_b2b') AS
SELECT 
di.invoice_id,
di.invoice_item_no,
di.product_id,
di.sales_order_id,
di.sales_order_item_no,
di.invoiced_quantity as brought_quantity,
di.primary_unit,
di.invoice_item_total_amount as item_total,
di.invoice_date,
so.distributor_id, 
so.order_date, 
-- so.expected_delivery_date, 
so.currency, 
-- so.product_id,  -- comment later
soi.order_quantity, 
soi.order_quantity_uom as unit_measure, 
soi.unit_price as selling_price, 
-- soi.item_total_amount
-- p.product_id,
p.product_name,
p.brand,
p.manufacturer,
p.category,
p.department,
p.unit_price as cost_price,
p.retail_price as mrp
FROM IDENTIFIER(catalog || '.' || silver_schema || '.distributor_invoice_mv') di 
LEFT JOIN IDENTIFIER(catalog || '.' || silver_schema || '.distributor_sale_order_mv') so on di.sales_order_id=so.order_id and di.sales_order_item_no=so.item_no
LEFT JOIN IDENTIFIER(catalog || '.' || silver_schema || '.product_mv') p on di.product_id=p.product_id;

In [0]:
CREATE OR REPLACE VIEW IDENTIFIER(catalog || '.' || gold_schema || '.product_b2c') AS
SELECT 
co.order_id,
co.order_item_id,
co.product_id,
co.quantity as brought_quantity,
co.unit_price as selling_price,
co.consumer_id,
co.order_date,
co.order_status,
co.total_price as item_total,
co.currency,
-- co.payment_method,
p.product_name,
p.brand,
p.manufacturer,
p.category,
p.department,
p.unit_price as cost_price,
p.retail_price as mrp
FROM IDENTIFIER(catalog || '.' || silver_schema || '.consumer_orders_mv') co
LEFT JOIN IDENTIFIER(catalog || '.' || silver_schema || '.product_mv') p on co.product_id=p.product_id
where co.order_status='Delivered';